# Gradient Boosting Classification — Theory & Interview Q&A

Gradient Boosting Classification is an ensemble learning method that builds models sequentially, each correcting the errors of the previous, using decision trees as weak learners.

| Aspect                | Details                                                                 |
|-----------------------|------------------------------------------------------------------------|
| **Definition**        | Sequentially builds models to minimize errors, using decision trees.     |
| **Equation**          | Combines weak learners by minimizing loss function                      |
| **Use Cases**         | Credit scoring, disease prediction, text classification                 |
| **Assumptions**       | Weak learners perform slightly better than random guessing              |
| **Pros**              | High accuracy, handles mixed data, flexible loss functions              |
| **Cons**              | Prone to overfitting, slower to train                                   |
| **Key Parameters**    | n_estimators, learning_rate, max_depth, subsample                      |
| **Evaluation Metrics**| Accuracy, Precision, Recall, F1-score                                   |

## Interview Q&A

**Q1: What is Gradient Boosting?**  
A: An ensemble method that builds models sequentially, each correcting previous errors.

**Q2: How does Gradient Boosting work?**  
A: It fits new models to the residuals of previous models.

**Q3: What are the advantages of Gradient Boosting?**  
A: High accuracy, flexible, handles mixed data types.

**Q4: What are the limitations?**  
A: Prone to overfitting, slower to train.

**Q5: How do you prevent overfitting in Gradient Boosting?**  
A: Use early stopping, limit tree depth, use subsampling.

**Q6: How do you evaluate Gradient Boosting?**  
A: Using accuracy, precision, recall, and F1-score.

In [ ]:
# 1️⃣ Import Libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# 2️⃣ Load Dataset
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

# 3️⃣ Split Dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4️⃣ Create Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Optional
    ('gb', GradientBoostingClassifier(random_state=42))
])

# 5️⃣ Hyperparameter Tuning
param_grid = {
    'gb__n_estimators': [100, 200],
    'gb__learning_rate': [0.01, 0.1, 0.2],
    'gb__max_depth': [3, 4, 5],
    'gb__min_samples_split': [2, 5, 10],
    'gb__min_samples_leaf': [1, 2, 4],
    'gb__max_features': ['sqrt', 'log2', None]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# 6️⃣ Evaluate Best Model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("Best Parameters:", grid_search.best_params_)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# 7️⃣ Feature Importance Visualization
importances = best_model.named_steps['gb'].feature_importances_
feat_imp_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feat_imp_df = feat_imp_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(8,5))
sns.barplot(x='Importance', y='Feature', data=feat_imp_df, palette='viridis')
plt.title("Feature Importance in Gradient Boosting Classifier")
plt.show()
